In [1]:
#importing the libraries
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.layers import Input, Embedding, LSTM , Dense,GlobalMaxPooling1D,Flatten
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt

In [2]:
#importing the dataset
with open('intents.json') as content:
  data1 = json.load(content)
#getting all the data to lists
tags = []
inputs = []
responses={}
for intent in data1['intents']:
  responses[intent['tag']]=intent['responses']
  for lines in intent['patterns']:
    inputs.append(lines)
    tags.append(intent['tag'])
#converting to dataframe
data = pd.DataFrame({"inputs":inputs,
                     "tags":tags})
print(data)

                              inputs           tags
0                                 Hi       greeting
1                        How are you       greeting
2                   Is anyone there?       greeting
3                              Hello       greeting
4                           Good day       greeting
5                           Whats up       greeting
6                                cya        goodbye
7                      See you later        goodbye
8                            Goodbye        goodbye
9                       I am Leaving        goodbye
10                   Have a Good day        goodbye
11             I would like to order          order
12              please take my order          order
13                   order something          order
14                             order          order
15                            coffee         coffee
16                   order me coffee         coffee
17            i want to order coffee         coffee
18     order

In [3]:
#removing punctuations
import string
data['inputs'] = data['inputs'].apply(lambda wrd:[ltrs.lower() for ltrs in wrd if ltrs not in string.punctuation])
data['inputs'] = data['inputs'].apply(lambda wrd: ''.join(wrd))
#tokenize the data
from tensorflow.keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(data['inputs'])
train = tokenizer.texts_to_sequences(data['inputs'])

#apply padding
from tensorflow.keras.preprocessing.sequence import pad_sequences
x_train = pad_sequences(train)

#encoding the outputs
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_train = le.fit_transform(data['tags'])

In [4]:
#input length
input_shape = x_train.shape[1]
print(input_shape)
#define vocabulary
vocabulary = len(tokenizer.word_index)
print("number of unique words : ",vocabulary)
#output length
output_length = le.classes_.shape[0]
print("output length: ",output_length)


6
number of unique words :  37
output length:  8


In [5]:
#creating the model
i = Input(shape=(input_shape,))
x = Embedding(vocabulary+1,10)(i)
x = LSTM(10,return_sequences=True)(x)
x = Flatten()(x)
x = Dense(output_length,activation="softmax")(x)
model  = Model(i,x)
#compiling the model
model.compile(loss="sparse_categorical_crossentropy",optimizer='adam',metrics=['accuracy'])
#training the model
train = model.fit(x_train,y_train,epochs=200)

Epoch 1/200
2/2 [==============================] - 1s 3ms/step - loss: 2.0823 - accuracy: 0.0600
Epoch 2/200
2/2 [==============================] - 0s 4ms/step - loss: 2.0807 - accuracy: 0.0797
Epoch 3/200
2/2 [==============================] - 0s 4ms/step - loss: 2.0792 - accuracy: 0.1501
Epoch 4/200
2/2 [==============================] - 0s 3ms/step - loss: 2.0780 - accuracy: 0.1501
Epoch 5/200
2/2 [==============================] - 0s 3ms/step - loss: 2.0770 - accuracy: 0.1397
Epoch 6/200
2/2 [==============================] - 0s 4ms/step - loss: 2.0756 - accuracy: 0.1501
Epoch 7/200
2/2 [==============================] - 0s 3ms/step - loss: 2.0748 - accuracy: 0.1501
Epoch 8/200
2/2 [==============================] - 0s 4ms/step - loss: 2.0739 - accuracy: 0.1501
Epoch 9/200
2/2 [==============================] - 0s 3ms/step - loss: 2.0729 - accuracy: 0.2102
Epoch 10/200
2/2 [==============================] - 0s 3ms/step - loss: 2.0721 - accuracy: 0.2702
Epoch 11/200
2/2 [===========

2/2 [==============================] - 0s 3ms/step - loss: 0.8081 - accuracy: 0.8799
Epoch 167/200
2/2 [==============================] - 0s 3ms/step - loss: 0.8022 - accuracy: 0.9203
Epoch 168/200
2/2 [==============================] - 0s 3ms/step - loss: 0.8047 - accuracy: 0.9099
Epoch 169/200
2/2 [==============================] - 0s 2ms/step - loss: 0.7931 - accuracy: 0.9099
Epoch 170/200
2/2 [==============================] - 0s 3ms/step - loss: 0.7858 - accuracy: 0.8799
Epoch 171/200
2/2 [==============================] - 0s 3ms/step - loss: 0.7801 - accuracy: 0.8499
Epoch 172/200
2/2 [==============================] - 0s 4ms/step - loss: 0.7672 - accuracy: 0.8499
Epoch 173/200
2/2 [==============================] - 0s 3ms/step - loss: 0.7609 - accuracy: 0.8499
Epoch 174/200
2/2 [==============================] - 0s 3ms/step - loss: 0.7551 - accuracy: 0.8499
Epoch 175/200
2/2 [==============================] - 0s 4ms/step - loss: 0.7510 - accuracy: 0.8499
Epoch 176/200
2/2 [=====

In [16]:
#chatting
import random
import string
while True:
  texts_p = []
  prediction_input = input('You : ')
  #removing punctuation and converting to lowercase
  prediction_input = [letters.lower() for letters in prediction_input if letters not in string.punctuation]
  prediction_input = ''.join(prediction_input)
  texts_p.append(prediction_input)
  #tokenizing and padding
  prediction_input = tokenizer.texts_to_sequences(texts_p)
  prediction_input = np.array(prediction_input).reshape(-1)
  prediction_input = pad_sequences([prediction_input],input_shape)
  #getting output from model
  output = model.predict(prediction_input)
  output = output.argmax()
  #finding the right tag and predicting
  response_tag = le.inverse_transform([output])[0]
  print("Going ChatBot : ",random.choice(responses[response_tag]))
  if response_tag == "goodbye":
    break

You : hi
Going ChatBot :  Hello!
You : medium burger
Going ChatBot :  burger_mid
You : a coffee
Going ChatBot :  coffee
You : goodbye
Going ChatBot :  Sad to see you go :(


In [7]:
import pickle
import joblib

In [10]:
model.save('intents_chatbot_tf2_lstm_order_food.h5')

In [11]:
joblib.dump(tokenizer,'token_chatbot.pkl')

['token_chatbot.pkl']

In [12]:
joblib.dump(le,'le_chatbot.pkl')

['le_chatbot.pkl']

In [15]:
joblib.dump(responses,'response_chatbot.pkl')

['response_chatbot.pkl']